<font color="green">*To start working on this notebook, or any other notebook that we will use in the Moringa Data Science Course, we will need to save our own copy of it. We can do this by clicking File > Save a Copy in Drive. We will then be able to make edits to our own copy of this notebook.*</font>

# Python Programming: Elastic Net Regression 

## Example

In [1]:
# Example 1
# ---
# Use the fair dataset from the pydataset library to predict marriage satisfaction based on the given variables.
# ---
# 
!pip install pydataset

     |████████████████████████████████| 15.9 MB 7.5 MB/s 
  Created wheel for pydataset: filename=pydataset-0.2.0-py3-none-any.whl size=15939432 sha256=85167d1b2bb576b42183facff7df7be6a366ee4140732aaa9b9a3846a2c6f39c
  Stored in directory: /root/.cache/pip/wheels/32/26/30/d71562a19eed948eaada9a61b4d722fa358657a3bfb5d151e2
Successfully built pydataset


In [2]:
# Importing our libraries
# 
from pydataset import data
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 5000)
pd.set_option('display.width', 10000)

from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

initiated datasets repo at: /root/.pydataset/


In [3]:
# Data preparation
# 
df=pd.DataFrame(data('Fair'))
df.loc[df.sex== 'male', 'sex'] = 0
df.loc[df.sex== 'female','sex'] = 1
df['sex'] = df['sex'].astype(int)
df.loc[df.child== 'no', 'child'] = 0
df.loc[df.child== 'yes','child'] = 1
df['child'] = df['child'].astype(int)
X=df[['religious','age','sex','ym','education','occupation','nbaffairs']]
y=df['rate']

In [4]:
# Creating our linear regression model for the purpose of comparison
# 
regression=LinearRegression()
regression.fit(X,y)
first_model=(mean_squared_error(y_true=y,y_pred=regression.predict(X)))
print(first_model) 

# This mean standard error score of 1.05 is our benchmark for determining 
# if the elastic net model will be better or worst. 

1.049873864469667


In [5]:
# Below are the coefficients of this first model. We use a for loop to go through 
# the model and the zip function to combine the two columns.
# 
coef_dict_baseline = {}
for coef, feat in zip(regression.coef_,X.columns):
    coef_dict_baseline[feat] = coef
coef_dict_baseline

{'age': -0.00905964542867384,
 'education': 0.06810255742293703,
 'nbaffairs': -0.07882571247653965,
 'occupation': -0.00597950685299818,
 'religious': 0.04235281110639179,
 'sex': 0.0888201333708709,
 'ym': -0.030458802565476555}

In [6]:
# Elastic Net Model
# Elastic net, just like ridge and lasso regression, requires normalize data. 
# This argument  is set inside the ElasticNet function. 
# The second thing we need to do is create our grid.
# 
elastic=ElasticNet(normalize=True)
search=GridSearchCV(estimator=elastic,param_grid={'alpha':np.logspace(-5,2,8),'l1_ratio':[.2,.4,.6,.8]},scoring='neg_mean_squared_error',n_jobs=1,refit=True,cv=10)

In [7]:
# We will now fit our model and display the best parameters and the best results we can get with that setup.
# 
search.fit(X,y)
search.best_params_
abs(search.best_score_)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_base.py:145: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), ElasticNet())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to original_alpha * np.sqrt(n_samples) if l1_ratio is 1, and to original_alpha * n_samples if l1_ratio is 0. For other values of l1_ratio, no analytic formula is available.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_base.py:145: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If

1.0819158709244472

In [8]:
# The best hyperparameters was an alpha set to 0.001 and a l1_ratio of 0.8. 
# With these settings we got an MSE of 1.08. This is above our baseline model of MSE 1.05  for the baseline model. 
# Which means that elastic net is doing worse than linear regression. 
# For clarity, we will set our hyperparameters to the recommended values and run on the data.
# 
elastic=ElasticNet(normalize=True,alpha=0.001,l1_ratio=0.75)
elastic.fit(X,y)
second_model=(mean_squared_error(y_true=y,y_pred=elastic.predict(X)))
print(second_model)

1.0566430678343806


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_base.py:145: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), ElasticNet())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to original_alpha * np.sqrt(n_samples) if l1_ratio is 1, and to original_alpha * n_samples if l1_ratio is 0. For other values of l1_ratio, no analytic formula is available.
  FutureWarning,


In [9]:
# Below are the coefficients
# 
coef_dict_baseline = {}
for coef, feat in zip(elastic.coef_,X.columns):
    coef_dict_baseline[feat] = coef
coef_dict_baseline

# The coefficients are mostly the same. 
# Notice that occupation was completely removed from the model in the elastic net version. 
# This means that this values was no good to the algorithm. Traditional regression cannot do this.

{'age': -0.008630896492807703,
 'education': 0.044290855954486376,
 'nbaffairs': -0.06679513627963517,
 'occupation': -0.0,
 'religious': 0.01947541724957858,
 'sex': 0.0181164645680909,
 'ym': -0.02422483127451295}

## Challenges

### <font color="green">Challenge 1</font>

In [ ]:
# Challenge 1
# ---
# Question: Using the given housiet, create a regression model to predict 
# the value of prices of a house using the given features. 
# ---
# Dataset url = http://bit.ly/BostonHousingDataset
# ---
# 
OUR CODE GOES HERE

### <font color="green">Challenge 2</font>

In [ ]:
# Challenge 2
# ---
# Question: Using the Ames Housing dataset, create a regression model to predict the sales price of home 
# applying elastic net regression.
# ---
# Dataset Source = http://bit.ly/HousePricesDataset
# 
OUR CODE GOES HERE

### <font color="green">Challenge 3</font>

In [ ]:
# Challenge 3
# ---
# Question: Given the medical cost personal dataset, accurately predict insurance cost using a regression model.
# ---
# Dataset Source = http://bit.ly/https://bit.ly/insurance-_dataset
# 
OUR CODE GOES HERE

### <font color="green">Challenge 4</font>

In [ ]:
# Challenge 4
# ---
# Question: Use ElasticNet regression to build a model that is able to accurately predict the profits of a startup.
# ---
# Dataset Source = http://bit.ly/StartupsDataset
# ---
# 
OUR CODE STARTS HERE

### <font color="green">Challenge 5</font>

In [ ]:
# Challenge 5
# ---
# Question: Build a prediction model to predict duration for any combination of country,operator, 
# services and category given the genre,language and number of units. 
# Apply ElasticNet regression while building your model. 
# ---
# Dataset Source = https://bit.ly/Audio_content_consumption
# ---
# 
OUR CODE STARTS HERE